In [16]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
import pandas as pd
import torchvision.transforms as transforms

In [3]:
# emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [6]:
# dataset loading
df = pd.read_csv('fer2013.csv')

In [7]:
df.isnull().sum()

,0
emotion,0
pixels,0
Usage,0


In [25]:
# function to pass pixels strings in numpy arrays
def string_to_image(pixel_string):
  pixels = np.array(pixel_string.split(' '), 'int32')
  image = np.reshape(pixels, (48, 48))
  return image

In [26]:
# preparing the data
X_train, y_train, X_test, y_test = [], [], [], []
for index, row in df.iterrows():
  image = string_to_image(row['pixels'])
  label = int(row['emotion'])
  usage = row['Usage']

  if usage == 'Training':
    X_train.append(image)
    y_train.append(label)
  elif usage == 'PublicTest' or usage == 'PrivateTest': # Combining test sets
    X_test.append(image)
    y_test.append(label)

In [27]:
X_train = np.array(X_train, 'float32')
y_train = np.array(y_train)
X_test = np.array(X_test, 'float32')
y_test = np.array(y_test)

In [28]:
# adding channel and dimension
X_train = np.expand_dims(X_train, 1)/255.0
X_test = np.expand_dims(X_test, 1)/255.0

In [29]:
# cobverting to pytorch tensors
X_train_tensor = torch.from_numpy(X_train)
y_train_tensor = torch.from_numpy(y_train).type(torch.LongTensor)
X_test_tensor = torch.from_numpy(X_test)
y_test_tensor = torch.from_numpy(y_test).type(torch.LongTensor)

In [30]:
# Datasets and Dataloader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [31]:
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [32]:
# defining the CNN model
class EmotionCNN(nn.Module):
  def __init__(self):
    super(EmotionCNN, self).__init__()
    self.conv_block1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2) # Output: 64x24x24
        )
    self.conv_block2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2) # Output: 128x12x12
        )
    self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 12 * 12, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 7) # 7 emotion classes
        )

  def forward(self, x):
    x = self.conv_block1(x)
    x = self.conv_block2(x)
    x = self.classifier(x)
    return x

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EmotionCNN().to(device)

In [34]:
# hyperparameters
learning_rate = 0.001
num_epochs = 30

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [35]:
# training loop
for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Debugging: Print the shape of the images tensor
        print(f"Epoch {epoch+1}, Batch {i+1}: Image tensor shape: {images.shape}")

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Streaming output truncated to the last 5000 lines.
Epoch 8, Batch 199: Image tensor shape: torch.Size([128, 1, 48, 48])
Epoch 8, Batch 200: Image tensor shape: torch.Size([128, 1, 48, 48])
Epoch 8, Batch 201: Image tensor shape: torch.Size([128, 1, 48, 48])
Epoch 8, Batch 202: Image tensor shape: torch.Size([128, 1, 48, 48])
Epoch 8, Batch 203: Image tensor shape: torch.Size([128, 1, 48, 48])
Epoch 8, Batch 204: Image tensor shape: torch.Size([128, 1, 48, 48])
Epoch 8, Batch 205: Image tensor shape: torch.Size([128, 1, 48, 48])
Epoch 8, Batch 206: Image tensor shape: torch.Size([128, 1, 48, 48])
Epoch 8, Batch 207: Image tensor shape: torch.Size([128, 1, 48, 48])
Epoch 8, Batch 208: Image tensor shape: torch.Size([128, 1, 48, 48])
Epoch 8, Batch 209: Image tensor shape: torch.Size([128, 1, 48, 48])
Epoch 8, Batch 210: Image tensor shape: torch.Size([128, 1, 48, 48])
Epoch 8, Batch 211: Image tensor shape: torch.Size([128, 1, 48, 48])
Epoch 8, Batch 212: Image tensor shape: torch.Size([

In [36]:
model.eval()

EmotionCNN(
  (conv_block1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
  

In [37]:
with torch.no_grad():
        n_correct = 0
        n_samples = 0
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()

        acc = 100.0 * n_correct / n_samples
        print(f'Accuracy on test set: {acc:.2f} %')

Accuracy on test set: 61.34 %


In [38]:
# Save the trained model
torch.save(model.state_dict(), 'emotion_cnn.pth')
print("Model trained and saved as emotion_cnn.pth")

Model trained and saved as emotion_cnn.pth
